## Linear Sys. Calc.

In [49]:
import ipywidgets as widgets
import numpy as np
from IPython.display import display, clear_output
from fractions import Fraction
from IPython.display import clear_output
import markdown 

# Grid container
grid_container = widgets.Output()

# Initial grid size
N = 3

# Matrix Value
init_matrix = [[0]*N for _ in range(N)]
result_matrix = np.zeros((N, N))

def create_grid(N):
    children = []
    for i in range(N):
        row = []
        for j in range(N):
            float_widget = widgets.FloatText(value=0, layout=widgets.Layout(width='50px'))  # Adjust width
            row.append(float_widget)
            children.append(float_widget)
            
            label_layout = widgets.Layout(width='auto', justify_content='center', align_items='center')  # Layout with red border for labels
            if j == N -1:
                children.append(widgets.Label(value=f'x{j+1} =', layout=label_layout))
                result_widget = widgets.FloatText(value=0, layout=widgets.Layout(width='50px'))  # Adjust width
                row.append(result_widget)
                children.append(result_widget)
            elif j < N - 1:
                children.append(widgets.Label(value=f'x{j+1} +', layout=label_layout))
            else:
                children.append(widgets.Label(value="", layout=label_layout))
            
        
        init_matrix[i] = row  # Update the init_matrix row with widget references

    return widgets.GridBox(
        children=children,
        layout=widgets.Layout(grid_template_columns="repeat({0}, 50px)".format((N)*2+1), grid_gap='2px', layout=widgets.Layout(margin_left='100px'))
    )

def expand_grid(button):
    global N
    N += 1
    init_matrix.append([0]*N)  # Expand init_matrix when grid expands
    with grid_container:
        clear_output(wait=True)
        display(create_grid(N))

def reduce_grid(button):
    global N
    N -= 1
    init_matrix.pop()  # Reduce init_matrix when grid reduces
    with grid_container:
        clear_output(wait=True)
        display(create_grid(N))


def display_markdown(input_text):
    print(input_text)
    html = markdown.markdown(input_text)
    display(HTML(html))

def solve(button):
    global result_matrix
    
    # Limpa o console
    result_matrix = np.zeros((N, N+1)) 
    
    # Access values from the widget references stored in init_matrix
    for i, row in enumerate(init_matrix):
        #print("i: ", i)
        for j, widget in enumerate(row):
            #print("row:", row)
            #print("j:", j)
            result_matrix[i, j] = widget.value
    Reducao()

# Button to expand grid, reduce grid
expand_button = widgets.Button(description="Expand Grid")
expand_button.on_click(expand_grid)
reduce_button = widgets.Button(description="Reduce Grid")
reduce_button.on_click(reduce_grid)

solve_button = widgets.Button(description="Solve!")
solve_button.on_click(solve)

output_text = widgets.HTML(
    value='',
    placeholder='Solution will appear here...',
    description='',
    layout=widgets.Layout(width='500px', height='300px')
)

# Display initial grid and button
with grid_container:
    display(create_grid(N))

display(grid_container, expand_button, reduce_button, solve_button,output_text)

Output()

Button(description='Expand Grid', style=ButtonStyle())

Button(description='Reduce Grid', style=ButtonStyle())

Button(description='Solve!', style=ButtonStyle())

HTML(value='', layout=Layout(height='300px', width='500px'), placeholder='Solution will appear here...')

Passo 1:
['1.00', '2.00', '3.00']
['4.00', '5.00', '6.00']

Troca da linha 1 com a linha 2:
['4.00', '5.00', '6.00']
['1.00', '2.00', '3.00']

Dividir a linha 1 pelo pivô 4:
['1.00', '1.25', '1.50']
['1.00', '2.00', '3.00']

Subtrair (1) vezes a linha 1 da linha 2:
['1.00', '1.25', '1.50']
['0.00', '0.75', '1.50']

Passo 2:
['1.00', '1.25', '1.50']
['0.00', '0.75', '1.50']

Troca da linha 2 com a linha 2:
['1.00', '1.25', '1.50']
['0.00', '0.75', '1.50']

Dividir a linha 2 pelo pivô 3/4:
['1.00', '1.25', '1.50']
['0.00', '1.00', '2.00']

Subtrair (5/4) vezes a linha 2 da linha 1:
['1.00', '0.00', '-1.00']
['0.00', '1.00', '2.00']

Resultado:
x1 = -1.00
x2 = 2.00


In [48]:
from fractions import Fraction

def print_matrix(matrix):
    for row in matrix:
        print([format_value(cell) for cell in row])
    print()

def print_matrix_area(matrix):
    for row in matrix:
        output_text.value += str([format_value(cell) for cell in row]) + "\n"

def format_value(value):
    if isinstance(value, float):
        decimal_places = len(str(value).split('.')[-1])
        if decimal_places > 2:
            return str(Fraction(value).limit_denominator())
    return format(value, ".2f")

def gauss_jordan(matrix):
    
    rows = len(matrix)
    columns = len(matrix[0]) - 1

    for i in range(rows):
        print(f"Passo {i+1}:")
        print_matrix(matrix)

        output_text.value += f"## Passo {i+1}:\n"
        print_matrix_area(matrix)

        # Encontrar o índice da linha pivô com o maior valor absoluto na coluna atual
        pivot_row = i
        for j in range(i + 1, rows):
            if abs(matrix[j][i]) > abs(matrix[pivot_row][i]):
                pivot_row = j

        # Trocar a linha atual pela linha pivô
        matrix[i], matrix[pivot_row] = matrix[pivot_row], matrix[i]
        print(f"Troca da linha {i+1} com a linha {pivot_row+1}:")
        print_matrix(matrix)
        output_text.value += f"Troca da linha {i+1} com a linha {pivot_row+1}:\n"
        print_matrix_area(matrix)

        # Reduzir a linha pivô para que o elemento na coluna atual seja igual a 1
        pivot = matrix[i][i]
        for j in range(columns + 1):
            matrix[i][j] /= pivot
        print(f"Dividir a linha {i+1} pelo pivô {str(Fraction(pivot).limit_denominator())}:")
        print_matrix(matrix)

        # Zerar os elementos nas outras linhas da coluna atual
        for j in range(rows):
            if j != i:
                factor = matrix[j][i]
                for k in range(columns + 1):
                    matrix[j][k] -= factor * matrix[i][k]
                print(f"Subtrair ({str(Fraction(factor).limit_denominator())}) vezes a linha {i+1} da linha {j+1}:")
                print_matrix(matrix)

    # Arredondar os valores das células com no máximo 2 casas decimais
 #   for i in range(rows):
 #       for j in range(columns + 1):
  #          matrix[i][j] = round(matrix[i][j], 2)

    return matrix


matrix = []

def Reducao():
	# Exemplo de matriz aumentada
    matrix = result_matrix.tolist()

	#print("Matriz inicial:")
	#print_matrix(matrix)
    result = gauss_jordan(matrix)

	# Resultado
    print("Resultado:")
    for i, row in enumerate(result):
        value = row[-1]
        print(f"x{i + 1} =", format_value(value))
                        
    output_text.value += "Resultado:\n"
    for i, row in enumerate(result):
        value = row[-1]
        output_text.value += f"x{i + 1} = {format_value(value)}\n"
    output_text.value = markdown.markdown(output_text.value)